In [42]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import html5lib
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold

In [4]:
def eval_con_mat(matrix):
    #use confusion matrix to compute accuracy (good calls divided by all calls made)
    print('Accuracy of {}%'.format(round(100*
                                         (matrix[1][1]+matrix[0][0])/
                                         (matrix[1][0]+matrix[0][1]+matrix[1][1]+matrix[0][0])),2))
    

    print('Sensitivity of {}%'.format(round(100*matrix[1][1]/(matrix[1][1] + matrix[1][0]),2)))
    print('Specificity of {}%'.format(round(100*matrix[0][0]/(matrix[0][0] + matrix[0][1]),2)))
    

In [5]:
#Read in a list of 2000+ positive words
pos_words_loc = ('https://github.com/gurkpet/Thinkful-Lessons/blob/master/'
             'Thinkful%202.2.7-Naive%20Bayes/Word%20Lists/positive-words.txt')
pos_words_list = pd.read_html(pos_words_loc, skiprows=35)

#clean up the read in by removing the first column of junk data
for row in pos_words_list:
   del row[0]    

#convert list into dataframe
pos_words = pd.DataFrame(pos_words_list[0])

In [6]:
list_pos_words = list(pos_words[1])

In [7]:
#read in amazon file
amzn_file_loc = ('https://raw.githubusercontent.com/gurkpet/Thinkful-Lessons/master/'
            'Thinkful%202.2.7-Naive%20Bayes/amazon_cells_labelled.txt')

df = pd.read_csv(amzn_file_loc, delimiter= '\t', header=None)
#rename column headers
amzn_dat = df
amzn_dat.columns = ['review', 'positive_review']
amzn_target = amzn_dat['positive_review']

In [8]:
for key in list_pos_words:
    amzn_dat[str(key)] = amzn_dat.review.str.contains(str(key), case = False)
amzn_dat['positive_review'] = (amzn_dat['positive_review'] == 1)
data_amzn = amzn_dat[list_pos_words]
amzn_target = amzn_dat['positive_review']

In [64]:
bnb = BernoulliNB()
skf = KFold(n_splits=5)
x= 0
for train, test in skf.split(data_amzn):
    x = x+1
    fold = data_amzn.ix[train]
    keep = data_amzn.ix[test]
    targ = amzn_target.ix[train]
    bnb = bnb.fit(data_amzn, amzn_target)
    pred = bnb.predict(keep)
    con_mat = confusion_matrix(amzn_target.ix[test],pred)
    print('\nIteration no. {} results'.format(x))
    eval_con_mat(con_mat)


Iteration no. 1 results
Accuracy of 88.0%
Sensitivity of 82.18%
Specificity of 93.94%

Iteration no. 2 results
Accuracy of 84.0%
Sensitivity of 79.09%
Specificity of 88.89%

Iteration no. 3 results
Accuracy of 80.0%
Sensitivity of 73.27%
Specificity of 86.87%

Iteration no. 4 results
Accuracy of 82.0%
Sensitivity of 72.16%
Specificity of 90.29%

Iteration no. 5 results
Accuracy of 84.0%
Sensitivity of 71.43%
Specificity of 94.5%


<h5>Is any of the data overfit?:</h5>
    Based on the similarities between our five iteriations it doesn't seem that our model is overfitting.

<h5>Which seem to perform the best? Why?:</h5>
    Iteration number 1 seems have have the best performance.  This is likely due to the skewness between the folds, where the data is more balanced between positive and negative reviews for the Training data for Iteration 1 than the rest of the iterations.  
    
<h5>What features seemed to be most impactful to performance?:</h5>
    ??????????????